In [1]:
import pandas as pd
import numpy as np

## Data Processing 

In [2]:
review_df=pd.read_csv('yelp_review.csv')

In [4]:
business_df=pd.read_csv('yelp_business.csv')

In [6]:
business_df['categories'] = business_df['categories'].map(lambda x: x.lower())

In [7]:
business_new=business_df[business_df['categories'].str.contains('restaurant')]

In [9]:
business_new.sort_values(['review_count'],ascending=False).head(5)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
97944,4JNXUYY8wbaaDmk3BPzlWw,"""Mon Ami Gabi""",The Strip,"""3655 Las Vegas Blvd S""",Las Vegas,NV,89109,36.112827,-115.172581,4.0,7361,1,french;steakhouses;restaurants;breakfast & brunch
119907,RESDUcs7fIiihp38-d6_6g,"""Bacchanal Buffet""",The Strip,"""3570 S Las Vegas Blvd""",Las Vegas,NV,89109,36.116113,-115.176222,4.0,7009,1,sandwiches;buffets;restaurants;breakfast & bru...
69993,K7lWdNUhCbcnEvI0NhGewg,"""Wicked Spoon""",The Strip,"""3708 Las Vegas Blvd S""",Las Vegas,NV,89109,36.109538,-115.176170,3.5,5950,1,buffets;breakfast & brunch;restaurants
81212,cYwJA2A6I12KNkm2rtXd5g,"""Gordon Ramsay BurGR""",The Strip,"""3667 Las Vegas Blvd S""",Las Vegas,NV,89109,36.110724,-115.172169,4.0,5447,0,american (traditional);burgers;restaurants
139699,DkYS3arLOhA8si5uUEmHOw,"""Earl of Sandwich""",The Strip,"""3667 Las Vegas Blvd S""",Las Vegas,NV,89109,36.108228,-115.171869,4.5,4869,1,caterers;sandwiches;restaurants;food delivery ...


In [11]:
review_ws=review_df[review_df['business_id']=='K7lWdNUhCbcnEvI0NhGewg']

In [13]:
review_ws=review_ws.reset_index()
del review_ws['index']

In [145]:
review_ws['positive'] = np.where(review_ws['stars']>3, 1, 0)

In [146]:
review_ws_positive=review_ws[review_ws['positive']==1].reset_index()
del review_ws_positive['index']

In [147]:
review_ws_negative=review_ws[review_ws['positive']==0].reset_index()
del review_ws_negative['index']

In [148]:
good_reviews=list(review_ws_positive['text'].values)
bad_reviews=list(review_ws_negative['text'].values)

## Positive Review TF-IDF

In [197]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
wd=stopwords.words()+['wicked','spoon','went','korean', 'would','give','make','sure','room','thai','tea','short','fried','chicken','wings','night','food','mac','cheese', 'chocolate','covered','strawberries','love','came','buffet','buffets','http','www', 'yelp', 'select','highly','recommend','good','like','best','worth','vegas','back','tried','year','crab','story','definitely','really']

vectorizer = TfidfVectorizer(ngram_range=(4,6),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4,
                             min_df=2, stop_words=wd)

In [198]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import string
lemmatizer = WordNetLemmatizer()
def lemmatize_sentence(sentence):
    words = word_tokenize(sentence)
    res_words = []
    for word in words:
        res_words.append(lemmatizer.lemmatize(word).strip(string.punctuation))
    return " ".join(res_words)

In [199]:
X = vectorizer.fit_transform(good_reviews)
terms = vectorizer.get_feature_names()
tf_idf= pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score["term"] = terms
score.sort_values(by="score", ascending=False, inplace=True)

In [200]:
scoregood=score.head(30)

In [201]:
scoregood

,score,term
try little bit everything,13.488639,try little bit everything
prime rib bone marrow,10.190567,prime rib bone marrow
prime rib leg lamb,10.104404,prime rib leg lamb
bone marrow first time,8.578799,bone marrow first time
located inside cosmopolitan hotel,7.978571,located inside cosmopolitan hotel
unlimited mimosas bloody mary,7.746044,unlimited mimosas bloody mary
bone marrow prime rib,7.243665,bone marrow prime rib
bottomless mimosas bloody mary,5.957487,bottomless mimosas bloody mary
buy one get one,5.534687,buy one get one
unlimited mimosas bloody marys,5.442783,unlimited mimosas bloody marys


In [202]:
scoregood.to_csv('good_reviews_tfidf.csv')

## Negative Review TF-IDF

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
wd=stopwords.words()+['wicked','spoon','korean', 'thai','tea','wish','home','could','years','ago','short','went','night','fried','chicken','wings','chocolate','covered','mac','cheese','strawberries','first','last','food','love','came','buffet','buffets','http','www', 'yelp', 'select','highly','recommend','good','like','best','worth','vegas','back','tried','year','everything','story','definitely','really']

vectorizer = TfidfVectorizer(ngram_range=(3,4),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.5,
                             min_df=2, stop_words=wd)

In [187]:
Y = vectorizer.fit_transform(bad_reviews)
terms = vectorizer.get_feature_names()
tf_idf1= pd.DataFrame(Y.toarray().transpose(), index=terms)
tf_idf1 = tf_idf1.sum(axis=1)
score1 = pd.DataFrame(tf_idf1, columns=["score"])
score1["term"] = terms
score1.sort_values(by="score", ascending=False, inplace=True)

In [188]:
scorebad=score1.head(30)

In [189]:
scorebad

,score,term
snow crab legs,11.588330,snow crab legs
king crab legs,8.959294,king crab legs
heard great things,6.264538,heard great things
sitting heat lamps,5.685229,sitting heat lamps
crab legs fresh,5.286545,crab legs fresh
bacchanal caesar palace,5.249821,bacchanal caesar palace
crab legs shrimp,4.884653,crab legs shrimp
seated right away,4.863317,seated right away
seafood crab legs,4.856738,seafood crab legs
crab legs small,4.822397,crab legs small


In [190]:
scorebad.to_csv('bad_reviews_tfidf.csv')